# **Bacterial Image Classifier**

### Libraries
*   Scikit-Image
> Image processing
*   Open Source Computer Vision
> SIFT and SURF
*   Scikit-Learn
> PCA
*   Tensorflow
> Deep learning Model


In [ ]:
import os
import cv2
import random
import skimage
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from skimage import io
from skimage import filters as ft
from skimage import exposure
from skimage import util
from skimage import feature
from skimage.morphology import disk
from sklearn.decomposition import PCA

get_ipython().run_line_magic('matplotlib', 'inline')

## Classes

### Display Images

In [ ]:
class DisplayImages:
    
    def saveImage(image, name):
        name = 'C:/Users/dell/Desktop/Enhance Image/'+name + '.tif'
        io.imsave(name, image, plugin='tifffile')
    
    def showImage(image, name = 'ok'):
        name = name + ' -> min: ' + str(np.min(image)) + ' max: ' + str(np.max(image))
        cv2.imshow(name , image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

### Dataset Labels

In [ ]:
class DatasetLabels:

    bacterialGenSep = ['Acinetobacter.baumanii','Actinomyces.israeli','Bacteroides.fragilis','Bifidobacterium.spp','Candida.albicans','Clostridium.perfringens','Enterococcus.faecalis','Enterococcus.faecium','Escherichia.coli','Fusobacterium']

    def getLabelFromImageName(imageName):
        imageName = imageName.replace('.tif','')  
        imageName = imageName.split('_')
        tempName = imageName[0]
        for ni in imageName[1:-1]:
            tempName += '.' + ni
        return DatasetLabels.getIndex(tempName)
    
    def getIndex(imageName):
        return DatasetLabels.bacterialGenSep.index(imageName)
    
    def getNameAt(index):
        return DatasetLabels.bacterialGenSep[index]

### Read Image

In [ ]:
class ReadImage:

    def readImageFrom(imagePath):
        return cv2.imread(imagePath, cv2.IMREAD_GRAYSCALE)

    def getAllImagesAddress(dirPath):
        imageNames = os.listdir(dirPath)
        random.shuffle(imageNames)
        return [os.path.join(dirPath, fname) for fname in imageNames], imageNames
        

### Preprocess Image

In [ ]:
class PreprocessImage:
    
    def inversOfImage(image):
        return util.invert(np.array(image, dtype='uint8'))
    
    def removeNoise(image):
        return ft.rank.median(image, skimage.morphology.disk(2))
    
    def rescaleIntensity(image):
        return exposure.rescale_intensity(image, out_range='uint8')
    
    def enhanceEdges(image):
        edgeExist = feature.canny(image, sigma=0.2)
        image[edgeExist]= np.max(image)
        return image
        

### Image Features

In [ ]:
class ImageFeatures:
    
    def siftExtractor(image):
        sift = cv2.xfeatures2d.SIFT_create(sigma = 1.6)
        keyPoints, descripter = sift.detectAndCompute(image, None)   
        return descripter

    def surfExtractor(image):
        surf = cv2.xfeatures2d.SURF_create()
        keyPoints, descripter = surf.detectAndCompute(image, None)        
        return descripter

    def reduceSiftFeatures(features):
        pca = PCA(n_components= 35)
        pcaComponents = np.array(pca.fit_transform(features.T))
        return pcaComponents
    
    def reduceSurfFeatures(features):
        pca = PCA(n_components= 4)
        pcaComponents = np.array(pca.fit_transform(features.T))
        print(pcaComponents.shape, end = ', ')
        return pcaComponents
    
    def getFeatrues(imagePath, featureExtractor):
        # Read-Image                
        image = ReadImage.readImageFrom(imagePath)
        
        # Pre-Process        
        image = PreprocessImage.inversOfImage(image)
        image = PreprocessImage.removeNoise(image)
        image = PreprocessImage.rescaleIntensity(image)
        image = PreprocessImage.enhanceEdges(image)
        
        if featureExtractor == 'SIFT':
            # SIFT Features
            imageFeatures = ImageFeatures.siftExtractor(image)
            imageFeatures = ImageFeatures.reduceSiftFeatures(imageFeatures)
        else:
            # SURF Features
            imageFeatures = ImageFeatures.surfExtractor(image)
            imageFeatures = ImageFeatures.reduceSurfFeatures(imageFeatures)
        
        return imageFeatures


In [ ]:
def getImageLabelFeatures(dirPath, featureExtractor):
    
    imagePaths, imageNames = ReadImage.getAllImagesAddress(dirPath)
    
    ImagesFeatureArray = []
    LabelsArray = []
    
    for i, imagePath in enumerate(imagePaths):        
        # Image Label
        imageLabel = DatasetLabels.getLabelFromImageName(imageNames[i])
        
        #Get Image Features
        imageFeatures = ImageFeatures.getFeatrues(imagePath, featureExtractor)
        
        # Add to Array
        ImagesFeatureArray.append(imageFeatures)
        LabelsArray.append(imageLabel)
        
    return ImagesFeatureArray, LabelsArray

In [ ]:
class ModelCallback(tf.keras.callbacks.Callback):   

    def on_epoch_end(self, epoch, logs={}):
        ACCURACY_THRESHOLD =  0.95
        if(logs.get('accuracy') > ACCURACY_THRESHOLD):
            print("\nReached %2.2f%% accuracy, so stopping training!!" %(ACCURACY_THRESHOLD*100))   
            self.model.stop_training = True

callbacks = ModelCallback()

### Deep Neural Network

In [ ]:
class DeepNeuralNetwork:

  def trainModel(xTrain, yTrain, xTest, yTest, featureExtractor):

    if featureExtractor == 'SIFT':
        model = tf.keras.models.Sequential([
            tf.keras.layers.Flatten(input_shape=(128, 35)),
            tf.keras.layers.Dense(7616, activation='relu'),
            tf.keras.layers.Dense(3808, activation='relu'),
            tf.keras.layers.Dense(3808, activation='relu'),
            tf.keras.layers.Dense(1904, activation='relu'),
            tf.keras.layers.Dropout(0.5),
            tf.keras.layers.Dense(10, activation='softmax')
        ])

        model.summary()
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = 0.005),
                            loss='sparse_categorical_crossentropy',
                            metrics=['accuracy'])
    else:

        model = tf.keras.models.Sequential([
            tf.keras.layers.Flatten(input_shape=(64, 4)),
            tf.keras.layers.Dense(4480, activation='relu'),
            tf.keras.layers.Dense(2240, activation='relu'),
            tf.keras.layers.Dense(2240, activation='relu'),
            tf.keras.layers.Dense(1120, activation='relu'),
            tf.keras.layers.Dropout(0.5),
            tf.keras.layers.Dense(10, activation='softmax')
        ])

        model.summary()
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = 0.005),
                            loss='sparse_categorical_crossentropy',
                            metrics=['accuracy'])
        
    history = model.fit(xtrain, ytrain, epochs=50, batch_size= 100, verbose=1, callbacks=[callbacks])
    model.evaluate(x_test, y_test)
    model.save("F:/UNIVERSITY/SMESTER 7/FYP/Dataset/RAW/"+featureExtractor+"rainedModel.h5")

    DeepNeuralNetwork.plotAccuracyGraph(history)

    def plotAccuracyGraph(history):
        accuracy = history.history['accuracy']
        loss = history.history['loss']
        epochs = range(len(accuracy))
        plt.plot(epochs, accuracy, 'r', label='Training accuracy')
        plt.title('Training Accuracy')
        plt.show()


### Main

In [ ]:
def Main(featureExtractor):

  TrainFileDir = 'F:/UNIVERSITY/SMESTER 7/FYP/Dataset/Croped/Train'
  xTrain, yTrain = getImageLabelFeatures(TrainFileDir, featureExtractor)
  
  TestFileDir = 'F:/UNIVERSITY/SMESTER 7/FYP/Dataset/Croped/Test'
  xTest, yTest = getImageLabelFeatures(TestFileDir, featureExtractor)

  DeepNeuralNetwork.trainModel(xTrain, yTrain, xTest, yTest, featureExtractor)


In [ ]:
featureExtractor = 'SIFT'
Main(featureExtractor)